# HPWREN Camera Metadata Process (Firemap API)

Summary: <br>
The HPWREN camera metadata is stored within the San Diego Supercomputer Center and is accessible via the FireMap API.

firemap api --> all cam metadata --> filter for hpwren only --> write to csv

Open Questions: <br>
- Are these active cameras only? (Are there ever historical cameras that are no longer active?)

## Get All Camera Metadata From Firemap API

In [1]:
import json
from datetime import datetime

import pandas as pd
import pytz
import requests

In [2]:
BASE_URL = "https://firemap.sdsc.edu/pylaski/"

In [3]:
# Get data for ALL cameras
# https://firemap.sdsc.edu/pylaski/stations?camera=only&selection=boundingBox&minLat=0&maxLat=90&minLon=-180&maxLon=0
entity = "stations"
payload = {
    "camera": "only",
    "selection": "boundingBox",
    "minLat": 0,
    "maxLat": 90,  # northern hemisphere
    "minLon": -180,
    "maxLon": 0,  # western hemisphere
}
response = requests.get(f"{BASE_URL}{entity}", params=payload)

In [4]:
if response.status_code in range(200, 300):  # successful response range
    response_json = response.json()
    features = response_json["features"]
else:
    # TODO:
    # Add error handling
    print("Error!")

In [5]:
cameras_df = pd.json_normalize(features)
print(f"Total cameras: {len(cameras_df['properties.description.id'])}")

Total cameras: 1157


In [6]:
# cameras_df

In [7]:
# Camera breakdown
# all camera metadata either has hpwren or axis prefix
cameras_df["properties.description.id"].str.split(r"_|-", expand=True)[0].str.extract(
    r"([a-zA-Z]+)"
)[0].str.lower().value_counts()

axis      1054
hpwren     103
Name: 0, dtype: int64

## Filter for HPWREN cameras only

In [8]:
# just get hpwren cameras (ignore axis)

hpwren_cameras_df = cameras_df[
    cameras_df["properties.description.id"]
    .str.lower()
    .str.contains("hpwren", regex=False)
].copy()

# dictoinary example below:
# hpwren_cameras = []
# for feat in features:
#     feat_id = feat["properties"]["description"]["id"]
#     if "hpwren" in feat_id.lower():
#         # print(feat_id)
#         hpwren_cameras.append(feat)

In [9]:
len(hpwren_cameras_df)

103

## Transformations

### Extract direction

In [10]:
# Extract direction from description
hpwren_cameras_df["direction"] = hpwren_cameras_df[
    "properties.description.id"
].str.split("_", n=1, expand=True)[1]

In [11]:
hpwren_cameras_df["direction"].value_counts()

north                26
east                 26
south                25
west                 25
unknown direction     1
Name: direction, dtype: int64

### Extract image name

In [12]:
# image name
hpwren_cameras_df["image_id"] = (
    hpwren_cameras_df["properties.latest-images"]
    .str[0]
    .str[0]
    .str["image"]
    .str.split("/")
    .str[-1]
    .str.split(".")
    .str[0]
)

In [13]:
hpwren_cameras_df.shape

(103, 31)

In [14]:
hpwren_cameras_df.head()

,type,geometry.type,geometry.coordinates,properties.description.name,properties.description.id,properties.description.url,properties.latest-images,properties.description.region,properties.description.is_patrol_mode,properties.description.tilt_current,...,properties.description.network,properties.description.ProdNbr,properties.description.isp,properties.description.fov_rt,properties.description.lastupdate,properties.description.ptz,properties.description.az_current,properties.description.attribution,direction,image_id
0,Feature,Point,"[-117.12, 32.55, 10]",,hpwren0_unknown direction,http://hpwren.ucsd.edu/cameras/TJE.html,[[{'image': 'http://hpwren.ucsd.edu/cameras/L/...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unknown direction,tje-1-mobo-c
1,Feature,Point,"[-116.8081, 33.1599, 4055]",Big Black Mountain,hpwren1_north,http://hpwren.ucsd.edu/cameras/BBlackMtn.html,[[{'image': 'http://hpwren.ucsd.edu/cameras/L/...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,north,bm-n-mobo-c
2,Feature,Point,"[-116.8081, 33.1599, 4055]",Big Black Mountain,hpwren1_east,http://hpwren.ucsd.edu/cameras/BBlackMtn.html,[[{'image': 'http://hpwren.ucsd.edu/cameras/L/...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,east,bm-e-mobo-c
3,Feature,Point,"[-116.8081, 33.1599, 4055]",Big Black Mountain,hpwren1_south,http://hpwren.ucsd.edu/cameras/BBlackMtn.html,[[{'image': 'http://hpwren.ucsd.edu/cameras/L/...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,south,bm-s-mobo-c
4,Feature,Point,"[-116.8081, 33.1599, 4055]",Big Black Mountain,hpwren1_west,http://hpwren.ucsd.edu/cameras/BBlackMtn.html,[[{'image': 'http://hpwren.ucsd.edu/cameras/L/...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,west,bm-w-mobo-c


### Extract long, lat, elevation

In [15]:
hpwren_cameras_df["long"] = hpwren_cameras_df["geometry.coordinates"].str[0]
hpwren_cameras_df["lat"] = hpwren_cameras_df["geometry.coordinates"].str[1]
hpwren_cameras_df["elevation"] = hpwren_cameras_df["geometry.coordinates"].str[2]

In [16]:
hpwren_cameras_df.shape

(103, 34)

In [17]:
hpwren_cameras_df.head()

,type,geometry.type,geometry.coordinates,properties.description.name,properties.description.id,properties.description.url,properties.latest-images,properties.description.region,properties.description.is_patrol_mode,properties.description.tilt_current,...,properties.description.fov_rt,properties.description.lastupdate,properties.description.ptz,properties.description.az_current,properties.description.attribution,direction,image_id,long,lat,elevation
0,Feature,Point,"[-117.12, 32.55, 10]",,hpwren0_unknown direction,http://hpwren.ucsd.edu/cameras/TJE.html,[[{'image': 'http://hpwren.ucsd.edu/cameras/L/...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,unknown direction,tje-1-mobo-c,-117.1200,32.5500,10
1,Feature,Point,"[-116.8081, 33.1599, 4055]",Big Black Mountain,hpwren1_north,http://hpwren.ucsd.edu/cameras/BBlackMtn.html,[[{'image': 'http://hpwren.ucsd.edu/cameras/L/...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,north,bm-n-mobo-c,-116.8081,33.1599,4055
2,Feature,Point,"[-116.8081, 33.1599, 4055]",Big Black Mountain,hpwren1_east,http://hpwren.ucsd.edu/cameras/BBlackMtn.html,[[{'image': 'http://hpwren.ucsd.edu/cameras/L/...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,east,bm-e-mobo-c,-116.8081,33.1599,4055
3,Feature,Point,"[-116.8081, 33.1599, 4055]",Big Black Mountain,hpwren1_south,http://hpwren.ucsd.edu/cameras/BBlackMtn.html,[[{'image': 'http://hpwren.ucsd.edu/cameras/L/...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,south,bm-s-mobo-c,-116.8081,33.1599,4055
4,Feature,Point,"[-116.8081, 33.1599, 4055]",Big Black Mountain,hpwren1_west,http://hpwren.ucsd.edu/cameras/BBlackMtn.html,[[{'image': 'http://hpwren.ucsd.edu/cameras/L/...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,west,bm-w-mobo-c,-116.8081,33.1599,4055


### Merge with manually entered camera metadata

- If manual data missing camera_name then merge
- If manual data has camera_name then union

In [18]:
cam_man_df = pd.read_csv("../../data/raw/camera_metadata_manual.csv")
cam_man_df = cam_man_df.add_suffix("_manual")

In [19]:
cam_man_df.shape

(131, 9)

In [20]:
cam_man_df.head()

,camera_id_manual,image_id_manual,camera_name_manual,direction_manual,gmap_lat_manual,gmap_long_manual,elevation_manual,x_resolution_manual,y_resolution_manual
0,hpwren0_unknown direction,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,hpwren1_north,NaN,NaN,NaN,33.159927,-116.808092,NaN,3072.0,2048.0
2,hpwren1_east,NaN,NaN,NaN,33.159927,-116.808092,NaN,3072.0,2048.0
3,hpwren1_south,NaN,NaN,NaN,33.159927,-116.808092,NaN,3072.0,2048.0
4,hpwren1_west,NaN,NaN,NaN,33.159927,-116.808092,NaN,3072.0,2048.0


In [21]:
cam_man_na_df = cam_man_df[cam_man_df["camera_name_manual"].isna()]

In [22]:
cam_man_na_df.shape

(103, 9)

In [23]:
hpwren_cameras_df = hpwren_cameras_df.merge(
    cam_man_na_df,
    left_on="properties.description.id",
    right_on="camera_id_manual",
    how="inner",
)

In [24]:
hpwren_cameras_df.shape

(103, 43)

In [25]:
cam_man_nona_df = cam_man_df[~cam_man_df["camera_name_manual"].isna()]

In [26]:
cam_man_nona_df.shape

(28, 9)

In [27]:
hpwren_cameras_df = pd.concat([hpwren_cameras_df, cam_man_nona_df])

In [28]:
hpwren_cameras_df.shape

(131, 43)

In [29]:
hpwren_cameras_df.head()

,type,geometry.type,geometry.coordinates,properties.description.name,properties.description.id,properties.description.url,properties.latest-images,properties.description.region,properties.description.is_patrol_mode,properties.description.tilt_current,...,elevation,camera_id_manual,image_id_manual,camera_name_manual,direction_manual,gmap_lat_manual,gmap_long_manual,elevation_manual,x_resolution_manual,y_resolution_manual
0,Feature,Point,"[-117.12, 32.55, 10]",,hpwren0_unknown direction,http://hpwren.ucsd.edu/cameras/TJE.html,[[{'image': 'http://hpwren.ucsd.edu/cameras/L/...,NaN,NaN,NaN,...,10.0,hpwren0_unknown direction,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Feature,Point,"[-116.8081, 33.1599, 4055]",Big Black Mountain,hpwren1_north,http://hpwren.ucsd.edu/cameras/BBlackMtn.html,[[{'image': 'http://hpwren.ucsd.edu/cameras/L/...,NaN,NaN,NaN,...,4055.0,hpwren1_north,NaN,NaN,NaN,33.159927,-116.808092,NaN,3072.0,2048.0
2,Feature,Point,"[-116.8081, 33.1599, 4055]",Big Black Mountain,hpwren1_east,http://hpwren.ucsd.edu/cameras/BBlackMtn.html,[[{'image': 'http://hpwren.ucsd.edu/cameras/L/...,NaN,NaN,NaN,...,4055.0,hpwren1_east,NaN,NaN,NaN,33.159927,-116.808092,NaN,3072.0,2048.0
3,Feature,Point,"[-116.8081, 33.1599, 4055]",Big Black Mountain,hpwren1_south,http://hpwren.ucsd.edu/cameras/BBlackMtn.html,[[{'image': 'http://hpwren.ucsd.edu/cameras/L/...,NaN,NaN,NaN,...,4055.0,hpwren1_south,NaN,NaN,NaN,33.159927,-116.808092,NaN,3072.0,2048.0
4,Feature,Point,"[-116.8081, 33.1599, 4055]",Big Black Mountain,hpwren1_west,http://hpwren.ucsd.edu/cameras/BBlackMtn.html,[[{'image': 'http://hpwren.ucsd.edu/cameras/L/...,NaN,NaN,NaN,...,4055.0,hpwren1_west,NaN,NaN,NaN,33.159927,-116.808092,NaN,3072.0,2048.0


In [30]:
# If original value null, then fill with new value

hpwren_cameras_df["camera_id"] = hpwren_cameras_df["properties.description.id"].fillna(
    hpwren_cameras_df["camera_id_manual"]
)

hpwren_cameras_df["camera_name"] = hpwren_cameras_df[
    "properties.description.name"
].fillna(hpwren_cameras_df["camera_name_manual"])

hpwren_cameras_df["image_id"] = hpwren_cameras_df["image_id"].fillna(
    hpwren_cameras_df["image_id_manual"]
)

hpwren_cameras_df["direction"] = hpwren_cameras_df["direction"].fillna(
    hpwren_cameras_df["direction_manual"]
)

hpwren_cameras_df["elevation"] = hpwren_cameras_df["elevation"].fillna(
    hpwren_cameras_df["elevation_manual"]
)

# If manual value null, then fill with orig value

hpwren_cameras_df["lat"] = hpwren_cameras_df["gmap_lat_manual"].fillna(
    hpwren_cameras_df["lat"]
)

hpwren_cameras_df["long"] = hpwren_cameras_df["gmap_long_manual"].fillna(
    hpwren_cameras_df["long"]
)

In [35]:
columns = [
    "camera_id",
    "direction",
    "camera_name",
    "image_id",
    "long",
    "lat",
    "elevation",
    "geometry.type",
    "geometry.coordinates",
    "x_resolution_manual",
    "y_resolution_manual",
    "properties.description.url",
    # "properties.latest-images",
]
hpwren_cameras_final_df = hpwren_cameras_df[columns]
hpwren_cameras_final_df

,camera_id,direction,camera_name,image_id,long,lat,elevation,geometry.type,geometry.coordinates,x_resolution_manual,y_resolution_manual,properties.description.url
0,hpwren0_unknown direction,unknown direction,,tje-1-mobo-c,-117.120000,32.550000,10.0,Point,"[-117.12, 32.55, 10]",NaN,NaN,http://hpwren.ucsd.edu/cameras/TJE.html
1,hpwren1_north,north,Big Black Mountain,bm-n-mobo-c,-116.808092,33.159927,4055.0,Point,"[-116.8081, 33.1599, 4055]",3072.0,2048.0,http://hpwren.ucsd.edu/cameras/BBlackMtn.html
2,hpwren1_east,east,Big Black Mountain,bm-e-mobo-c,-116.808092,33.159927,4055.0,Point,"[-116.8081, 33.1599, 4055]",3072.0,2048.0,http://hpwren.ucsd.edu/cameras/BBlackMtn.html
3,hpwren1_south,south,Big Black Mountain,bm-s-mobo-c,-116.808092,33.159927,4055.0,Point,"[-116.8081, 33.1599, 4055]",3072.0,2048.0,http://hpwren.ucsd.edu/cameras/BBlackMtn.html
4,hpwren1_west,west,Big Black Mountain,bm-w-mobo-c,-116.808092,33.159927,4055.0,Point,"[-116.8081, 33.1599, 4055]",3072.0,2048.0,http://hpwren.ucsd.edu/cameras/BBlackMtn.html
...,...,...,...,...,...,...,...,...,...,...,...,...
106,NaN,west,Santiago Peak,stgo-w-mobo-c,-117.534115,33.711172,5669.0,NaN,NaN,3072.0,2048.0,NaN
127,NaN,north,White Star,ws-n-mobo-c,-116.318014,32.647266,4000.0,NaN,NaN,3072.0,2048.0,NaN
128,NaN,east,White Star,ws-e-mobo-c,-116.318014,32.647266,4000.0,NaN,NaN,3072.0,2048.0,NaN
129,NaN,south,White Star,ws-s-mobo-c,-116.318014,32.647266,4000.0,NaN,NaN,3072.0,2048.0,NaN


## Write camera metadata to raw and processed folders

In [36]:
# current_time_str = datetime.now(tz=pytz.UTC).strftime("%m/%d/%Y, %H:%M:%S")
current_time_str = datetime.now(tz=pytz.UTC).strftime("%Y%m%d_%H_%M_%S")

# all cameras
raw_output_path = f"../../data/raw/camera_metadata.csv"
cameras_df.to_csv(raw_output_path, index=False)

# hpwren cameras only
processed_output_path = f"../../data/processed/camera_metadata_hpwren.csv"
hpwren_cameras_final_df.to_csv(processed_output_path, index=False)